In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")

In [ ]:
train_data.head()

In [11]:
X_test_full = pd.read_csv("/kaggle/input/titanic/test.csv")

In [13]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier

y = train_data['Survived']
features = ['Pclass', 'SibSp', 'Sex', 'Parch']

X = pd.get_dummies(train_data[features])
X['Age'] = train_data['Age']



def get_mae(max_leaf_nodes,train_X, val_X, train_y, test_y):
    model = RandomForestClassifier(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    pred_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, pred_val)
    return mae

In [14]:
from sklearn.model_selection import train_test_split

In [29]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, random_state=0)
min_mae = 0

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].dtype == "object"]

my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()


In [31]:
X_test = pd.get_dummies(test_data[features])
X_test["Age"] = test_data["Age"]

'''model = RandomForestClassifier(max_leaf_nodes=min_nodes, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({"PassengerId": test_data.PassengerId, 'Survived': predictions})
output.to_csv('first_submission.csv', index=False)
print("Ok")'''

'model = RandomForestClassifier(max_leaf_nodes=min_nodes, max_depth=5, random_state=1)\nmodel.fit(X, y)\npredictions = model.predict(X_test)\n\noutput = pd.DataFrame({"PassengerId": test_data.PassengerId, \'Survived\': predictions})\noutput.to_csv(\'first_submission.csv\', index=False)\nprint("Ok")'

In [32]:
numerical_transformer = SimpleImputer(strategy='mean') # Your code here

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
]) # Your code here

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestClassifier()

In [33]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])
my_pipeline.fit(X_train, y_train)
preds = my_pipeline.predict(X_valid)
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 0.31390134529147984


In [34]:
preds_test = my_pipeline.predict(X_test)

/opt/conda/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:440: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


In [36]:
output = pd.DataFrame({'PassengerId': X_test_full.PassengerId,
                       'Survived': preds_test})
output.to_csv('submission.csv', index=False)

AttributeError: 'DataFrame' object has no attribute 'PassengerId'